In [290]:
import pandas as pd
import glob

path = "LoLesports_data"

all_files = glob.glob(path + "/*.csv")

df_list = []

for file in all_files[-3:]:
    temp_df = pd.read_csv(file, low_memory=False)
    df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)
df = df[df["datacompleteness"] == "complete"]
df.drop(columns=["datacompleteness", "url", "year"], inplace=True)

train = df[(df["patch"] >= 12.01) & (df["patch"] <= 14.05)]
test = df[df["patch"] >= 14.06]

train.shape, test.shape

((266040, 158), (67380, 158))

In [297]:
teams = train[train["position"] == "team"].copy()
teams.drop(columns=["champion", "position"], inplace=True)
teams.shape

(44340, 156)

In [292]:
games_per_patch = teams.groupby("patch").size() / 2

ban_list = ["ban1", "ban2", "ban3", "ban4", "ban5"]
agg_list = {ban: "value_counts" for ban in ban_list}

tmp = teams.groupby("patch").agg(agg_list).fillna(0)
ban_sum = tmp[ban_list].sum(axis=1)

ban_rate = (ban_sum / games_per_patch).sort_index()
ban_rate = (ban_rate * 100).round(2)

df = pd.DataFrame({
    'ban_count': ban_sum,
    'ban_rate': ban_rate,
})

df.head()

ban_count  ban_rate
patch                             
12.01 Aatrox        15.0      2.15
      Ahri           4.0      0.57
      Akali        224.0     32.14
      Akshan        63.0      9.04
      Alistar        6.0      0.86

In [293]:
pick_list = ["pick1", "pick2", "pick3", "pick4", "pick5"]
agg_list = {pick: "value_counts" for pick in pick_list}

tmp = teams.groupby("patch").agg(agg_list).fillna(0)
pick_sum = tmp[pick_list].sum(axis=1)

pick_rate = (pick_sum / games_per_patch).sort_index()
pick_rate = (pick_rate * 100).round(2)

ban_pick_count = ban_sum + pick_sum
ban_pick_rate = (ban_pick_count / games_per_patch).sort_index()
ban_pick_rate = (ban_pick_rate * 100).round(2)

tmp = pd.DataFrame({
    "pick_count": pick_sum,
    "pick_rate": pick_rate,
    "ban_pick_count": ban_pick_count,
    "ban_pick_rate": ban_pick_rate,
})

df = pd.concat([df, tmp], axis=1)
df.fillna(0, inplace=True)
df.head()

ban_count  ban_rate  pick_count  pick_rate  ban_pick_count  \
patch                                                                       
12.01 Aatrox        15.0      2.15        22.0       3.16            37.0   
      Ahri           4.0      0.57         0.0       0.00             0.0   
      Akali        224.0     32.14        99.0      14.20           323.0   
      Akshan        63.0      9.04        23.0       3.30            86.0   
      Alistar        6.0      0.86        30.0       4.30            36.0   

               ban_pick_rate  
patch                         
12.01 Aatrox            5.31  
      Ahri              0.00  
      Akali            46.34  
      Akshan           12.34  
      Alistar           5.16

In [294]:
melted = teams.melt(id_vars=['patch', 'result'], value_vars=pick_list, var_name='pick_order', value_name='champion')
tmp = melted.groupby(['patch', 'champion']).agg(
    wins=('result', 'sum'),
    games=('result', 'count'),
    win_rate=("result", "mean")
)

df = pd.concat([df, tmp], axis=1)
df.head()

ban_count  ban_rate  pick_count  pick_rate  ban_pick_count  \
patch                                                                       
12.01 Aatrox        15.0      2.15        22.0       3.16            37.0   
      Ahri           4.0      0.57         0.0       0.00             0.0   
      Akali        224.0     32.14        99.0      14.20           323.0   
      Akshan        63.0      9.04        23.0       3.30            86.0   
      Alistar        6.0      0.86        30.0       4.30            36.0   

               ban_pick_rate  wins  games  win_rate  
patch                                                
12.01 Aatrox            5.31  11.0   22.0  0.500000  
      Ahri              0.00   NaN    NaN       NaN  
      Akali            46.34  52.0   99.0  0.525253  
      Akshan           12.34  12.0   23.0  0.521739  
      Alistar           5.16  20.0   30.0  0.666667

In [295]:
df.fillna(0, inplace=True)
df.reset_index(inplace=True)
df.to_csv("output/champions_stat.csv", index=False)